### Introducción a la tokenización y los tokenizadores

La **tokenización** es el proceso mediante el cual se divide un texto en unidades más pequeñas llamadas **tokens**. Estas unidades pueden ser palabras, caracteres o subpalabras. El objetivo principal de la tokenización es transformar un texto complejo en componentes manejables que puedan ser procesados por algoritmos y modelos de aprendizaje profundo. Esta transformación es esencial para que la computadora pueda interpretar y analizar el lenguaje humano, ya que los modelos no trabajan directamente con texto, sino con representaciones numéricas derivadas de estos tokens.

Un **tokenizador** es la herramienta o algoritmo encargado de llevar a cabo este proceso de segmentación. Al tokenizar el texto, el tokenizador asigna a cada unidad (token) un identificador numérico que posteriormente se utiliza en las tareas de procesamiento, ya sea para entrenamiento de modelos de lenguaje, análisis de sentimientos, traducción automática, chatbots y muchas otras aplicaciones. La forma en que se lleva a cabo la tokenización puede influir de forma directa en la capacidad del modelo para aprender y capturar patrones lingüísticos, por lo que es crucial escoger el enfoque adecuado según el problema y el idioma.


#### **Configuraciones**


Para este cuaderno, utilizarás las siguientes librerías:

* [**nltk**](https://www.nltk.org/) o Natural Language Toolkit, se empleará para tareas de gestión de datos. Ofrece herramientas y recursos integrales para procesar texto en lenguaje natural, lo que la hace una opción valiosa para tareas como el preprocesamiento y análisis de texto.

* [**spaCy**](https://spacy.io/) es una librería de software de código abierto para el procesamiento avanzado del lenguaje natural en Python. spaCy es reconocido por su velocidad y precisión al procesar grandes volúmenes de datos textuales.

* [**BertTokenizer**](https://huggingface.co/docs/transformers/main_classes/tokenizer#berttokenizer) forma parte de la librería Hugging Face Transformers. BertTokenizer está diseñado específicamente para tokenizar texto según las especificaciones del modelo BERT.

* [**XLNetTokenizer**](https://huggingface.co/docs/transformers/main_classes/tokenizer#xlnettokenizer) es otro componente de la librería Hugging Face Transformers. Está adaptado para tokenizar texto de acuerdo con los requerimientos del modelo XLNet.

* [**torchtext**](https://pytorch.org/text/stable/index.html) es parte del ecosistema de PyTorch, para manejar diversas tareas de procesamiento del lenguaje natural. Simplifica el proceso de trabajar con datos textuales y provee funcionalidades para el preprocesamiento de datos, tokenización, gestión de vocabulario y agrupación en lotes.


#### Instalación de librerías requeridas


In [ ]:
#!pip install nltk
#!pip install transformers
#!pip install sentencepiece
#!pip install spacy
#!pip install numpy==1.24
#!python -m spacy download en_core_web_sm
#!python -m spacy download de_core_news_sm
#!pip install numpy scikit-learn
#!pip install torch==2.0.1
#!pip install torchtext==0.15.2

#### Importación de librerías requeridas

_Se recomienda importar todas las librerías requeridas en un solo lugar (aquí):_


In [ ]:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
import spacy
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.util import ngrams
from transformers import BertTokenizer
from transformers import XLNetTokenizer

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#### ¿Qué es un tokenizador y por qué lo usamos?

Los tokenizadores juegan un papel fundamental en el procesamiento del lenguaje natural, segmentando el texto en unidades más pequeñas conocidas como tokens. Estos tokens se transforman posteriormente en representaciones numéricas llamadas índices de tokens, que son utilizados directamente por los algoritmos de aprendizaje profundo.
<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%201.png" width="50%" alt="Image Description">
</center>


#### Tipos de tokenizador

Existen varios tipos de tokenizadores que se pueden clasificar según la unidad en la que se divida el texto. Los enfoques principales son:

1. **Tokenización basada en palabras:** Separa el texto dividiéndolo en palabras individuales.
2. **Tokenización basada en caracteres:** Cada carácter individual se convierte en un token.
3. **Tokenización basada en subpalabras:** Combina lo mejor de los dos métodos anteriores, manteniendo palabras frecuentes intactas y dividiendo aquellas menos comunes en unidades sublexicales o subpalabras.

Cada uno de estos enfoques tiene sus ventajas y limitaciones, lo que determina su aplicación en diferentes contextos.

<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%202.png" width="50%" alt="Image Description">
</center>


#### Tokenizador basado en palabras
La tokenización basada en palabras es uno de los métodos más sencillos y utilizados. En este enfoque, el texto se divide en palabras siguiendo reglas específicas que pueden incluir la separación por espacios en blanco o puntuación. 

**Uso de NLTK:**  

La librería NLTK (Natural Language Toolkit) es una de las herramientas clásicas para el procesamiento de lenguaje natural en Python. Utiliza funciones como `word_tokenize` para dividir el texto en palabras. Por ejemplo:

In [ ]:
texto = "This is a sample sentence for word tokenization."
tokens = word_tokenize(texto)
print(tokens)

Este fragmento de código toma la cadena de texto y la descompone en tokens que corresponden a palabras y signos de puntuación. Cabe destacar que NLTK aplica reglas propias para tratar contracciones. Por ejemplo, en el siguiente código:


In [ ]:
# Esto muestra word_tokenize de la librería nltk

texto = "I couldn't help the dog. Can't you do it? Don't be afraid if you are."
tokens = word_tokenize(texto)
print(tokens)

**Uso de spaCy y torchtext:**  
Otra opción es utilizar spaCy, una librería reconocida por su velocidad y precisión en el procesamiento de grandes volúmenes de texto. En combinación con torchtext, se puede obtener una lista de tokens y además analizar detalles como la parte de la oración (POS) o las dependencias sintácticas. Un ejemplo de código es:

In [ ]:
# Esto muestra el uso del tokenizador de 'spaCy' con la función get_tokenizer de torchtext

texto = "I couldn't help the dog. Can't you do it? Don't be afraid if you are."
nlp = spacy.load("en_core_web_sm")
doc = nlp(texto)

# Creando una lista de tokens e imprimiéndola
token_list = [token.text for token in doc]
print("Tokens:", token_list)

# Mostrando detalles de cada token
for token in doc:
    print(token.text, token.pos_, token.dep_)

En este código, se carga el modelo de lenguaje de spaCy, se procesa el texto y se extraen tanto los tokens como sus etiquetas gramaticales y dependencias sintácticas. Esto resulta muy útil para entender el rol que cumple cada palabra en la oración y para realizar análisis lingüísticos más profundos.

Explicación de algunas líneas:

- I PRON nsubj: "I" es un pronombre (PRON) y es el sujeto nominal (nsubj) de la oración.
- help VERB ROOT: "help" es un verbo (VERB) y es la acción principal (ROOT) de la oración.
- afraid ADJ acomp: "afraid" es un adjetivo (ADJ) y es un complemento adjetival (acomp) que aporta más información sobre un estado o cualidad relacionado con el verbo.


> Librerías generales como nltk y spaCy a menudo dividen palabras como "don't" y "couldn't", que son contracciones, en palabras individuales separadas. **No existe una regla universal, y cada librería tiene sus propias reglas de tokenización para tokenizadores basados en palabras**. 

> Sin embargo, la pauta general es preservar el formato de entrada después de la tokenización para que coincida con la forma en que se entrenó el modelo.

El problema con este algoritmo es que las palabras con significados similares serán asignadas con IDs diferentes, lo que resulta en que se traten como palabras completamente separadas con significados distintos. 

Por ejemplo, *Unicorns* es la forma plural de *Unicorn*, pero un tokenizador basado en palabras las tokenizaría como dos palabras separadas, lo que podría causar que el modelo no reconozca su relación semántica.


In [ ]:
texto = "Unicorns are real. I saw a unicorn yesterday."
token = word_tokenize(texto)
print(token)

Cada palabra se divide en un token, lo que conduce a un aumento significativo en el vocabulario total del modelo. Cada token se asigna a un vector  que contiene los significados de la palabra, resultando en parámetros de modelo grandes.

Los idiomas generalmente tienen una gran cantidad de palabras, por lo que los vocabularios basados en ellas siempre serán extensos. Sin embargo, el número de caracteres en un idioma siempre es menor en comparación con el número de palabras. 


#### Tokenización basado en caracteres

La **tokenización basada en caracteres** implica dividir el texto en caracteres individuales. Una de sus principales ventajas es que los vocabularios resultantes son **intrínsecamente pequeños**. Además, dado que los idiomas suelen tener un conjunto limitado de caracteres, el número de tokens fuera del vocabulario (**out-of-vocabulary tokens**) también se reduce, lo que minimiza el desperdicio de tokens.

**Por ejemplo:**  

Texto de entrada: `This is a sample sentence for tokenization.`  

Salida de tokenización basada en caracteres:  

`['T', 'h', 'i', 's', 'i', 's', 'a', 's', 'a', 'm', 'p', 'l', 'e', 's', 'e', 'n', 't', 'e', 'n', 'c', 'e', 'f', 'o', 'r', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', '.']`

Sin embargo, es importante tener en cuenta que este enfoque también tiene **limitaciones**. Los caracteres individuales no capturan el mismo nivel de información semántica que las palabras completas, y la longitud total de la secuencia de tokens aumenta considerablemente. Esto puede afectar el **tamaño del modelo**, incrementar el tiempo de entrenamiento y, en algunos casos, reducir el rendimiento del sistema.


#### Tokenizador basado en subpalabras

El **tokenizador basado en subpalabras** permite que las palabras de uso frecuente permanezcan intactas, mientras que descompone las palabras poco frecuentes en **subunidades significativas**. Técnicas como [SentencePiece](https://github.com/google/sentencepiece) o [WordPiece](https://paperswithcode.com/method/wordpiece) se utilizan comúnmente para este propósito. Estos métodos aprenden unidades de subpalabras a partir de un corpus de texto, identificando prefijos, sufijos y raíces comunes como tokens de subpalabras, basándose en su **frecuencia de aparición**.

Este enfoque ofrece la ventaja de representar una gama más amplia de palabras y adaptarse a los **patrones lingüísticos específicos** de un corpus, lo que mejora la capacidad del modelo para generalizar y manejar palabras no vistas.

En los ejemplos que se muestran a continuación, las palabras se dividen en subpalabras, lo cual ayuda a preservar la **información semántica** de la palabra completa. Por ejemplo, *"Unhappiness"* se divide en 
*"un"* y *"happiness"*, que pueden aparecer como subpalabras independientes. Al combinarlas nuevamente, se reconstruye *"unhappiness"*, manteniendo su significado contextual. Este enfoque permite conservar tanto la estructura como el sentido semántico de las palabras.

<center>  
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%203.png" width="50%" alt="Image Description">  
</center>


#### WordPiece

WordPiece es un algoritmo que se utiliza para la tokenización de subpalabras y se implementa, por ejemplo, en el tokenizador BertTokenizer. Su funcionamiento se basa en iniciar el vocabulario con todos los caracteres presentes en el corpus de entrenamiento y, posteriormente, fusionar pares de caracteres o subpalabras de manera progresiva. En lugar de seleccionar el par más frecuente, WordPiece elige el par que, al fusionarse, maximiza la probabilidad de los datos de entrenamiento. Este criterio permite que el modelo decida cuáles combinaciones de caracteres aportan más información semántica.

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize("IBM taught me tokenization.")

A continuación, se muestra un desglose de la salida:
- 'ibm': "IBM" se tokeniza como 'ibm'. BERT convierte los tokens a minúsculas, ya que no conserva la información de mayúsculas cuando se utiliza el modelo "bert-base-uncased".
- 'taught', 'me', '.': Estos tokens son iguales a las palabras o puntuaciones originales, solo que en minúsculas (excepto la puntuación).
- 'token', '##ization': "Tokenization" se divide en dos tokens. "Token" es una palabra completa, y "##ization" es una parte de la palabra original. El "##" indica que "ization" debe conectarse de nuevo a "token" al realizar la detokenización (transformar tokens de vuelta a palabras).


#### Unigram y SentencePiece

El método **Unigram** trabaja de manera distinta a WordPiece. En este enfoque, se parte de una lista amplia de posibles subpalabras y se reduce progresivamente dicha lista según la frecuencia con la que aparecen estas unidades en el texto. Se selecciona el conjunto de subpalabras que mejor representa los datos, logrando una tokenización eficiente y flexible.

**SentencePiece** es una herramienta que implementa el enfoque Unigram dentro de un marco más amplio para la tokenización de subpalabras. Esta herramienta procesa el texto, lo segmenta en subpalabras y asigna a cada segmento un identificador (ID) de manera consistente. Una de sus ventajas es la reproducibilidad: al aplicar SentencePiece sobre el mismo conjunto de datos, se obtienen las mismas subpalabras e IDs en cada ejecución. Esto es esencial para tareas donde se requiere que la tokenización sea consistente entre diferentes fases del procesamiento del lenguaje.


In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
tokenizer.tokenize("IBM taught me tokenization.")

Esto es lo que sucede con cada token:
- '▁IBM': El "▁" (a menudo referido como "carácter de espacio") antes de "IBM" indica que este token está precedido por un espacio en el texto original. "IBM" se mantiene tal cual porque es reconocido como un token completo por XLNet y preserva la capitalización, ya que se está utilizando el modelo "xlnet-base-cased".
- '▁taught', '▁me', '▁token': De manera similar, estos tokens están prefijados con "▁" para indicar que son palabras nuevas precedidas por un espacio en el texto original, preservando la palabra completa y manteniendo la capitalización original.
- 'ization': A diferencia de "BertTokenizer", "XLNetTokenizer" no utiliza "##" para indicar tokens de subpalabras. "ization" aparece como un token propio sin prefijo porque sigue directamente a la palabra anterior "token" sin un espacio en el texto original.
- '.': El punto se tokeniza como un token separado ya que la puntuación se trata por separado.


#### Tokenización con PyTorch 

El procesamiento del lenguaje natural (NLP) requiere que el texto, originalmente legible para los humanos, se convierta en un formato numérico que los modelos de aprendizaje profundo puedan manipular. En este contexto, la **tokenización** es el primer paso fundamental. La tokenización consiste en descomponer un texto en unidades más pequeñas, denominadas **tokens**. Estos tokens pueden ser palabras, subpalabras o incluso caracteres. 

En PyTorch, la librería **torchtext** proporciona herramientas que simplifican este proceso, permitiendo convertir secuencias de texto en representaciones numéricas mediante la asignación de índices a cada token.
La ventaja principal de este proceso es que los algoritmos de NLP operan de manera eficiente sobre números. Una vez que se han asignado índices a cada token, los modelos pueden utilizar estos valores para aprender patrones, realizar inferencias y llevar a cabo tareas como clasificación, traducción o análisis de sentimientos.




**Conjunto de datos y preparación**

Para ilustrar el proceso, se utiliza un conjunto de datos definido como:

In [ ]:
dataset = [
    (1,"Introduction to NLP"),
    (2,"Basics of PyTorch"),
    (1,"NLP Techniques for Text Classification"),
    (3,"Named Entity Recognition with PyTorch"),
    (3,"Sentiment Analysis using PyTorch"),
    (3,"Machine Translation with PyTorch"),
    (1," NLP Named Entity,Sentiment Analysis,Machine Translation "),
    (1," Machine Translation with NLP "),
    (1," Named Entity vs Sentiment Analysis  NLP ")]


Dentro del ecosistema de PyTorch, **torchtext** ofrece una función muy útil llamada `get_tokenizer`, que permite obtener tokenizadores predefinidos según el método deseado. Por ejemplo, al solicitar el tokenizador `"basic_english"`, se obtiene una función que normaliza las cadenas (convierte a minúsculas, elimina ciertos caracteres especiales, etc.) y luego realiza una división simple basada en espacios. Este método es adecuado para muchos casos en los que no se requiere un análisis gramatical muy complejo.

El siguiente fragmento de código muestra cómo se importa y se utiliza esta función:

```python
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")
```

Con la línea anterior se obtiene una función tokenizadora que se puede aplicar a cualquier cadena de texto. Por ejemplo, al tokenizar el primer elemento del conjunto de datos, se obtiene:

```python
tokenizer(dataset[0][1])
```

Si el texto es "Introduction to NLP", el tokenizador lo convertirá en una lista de tokens como `['introduction', 'to', 'nlp']` (notar que se normaliza a minúsculas).



Cada elemento del dataset es una tupla en la que el primer elemento puede representar una etiqueta o identificador (por ejemplo, el ID de una categoría) y el segundo es una cadena de texto que se desea tokenizar.

Para procesar este conjunto de datos, se utiliza una **función generadora** denominada `yield_tokens`. Esta función permite recorrer el dataset y, de forma perezosa (lazy), tokenizar cada cadena a medida que se solicita. Este enfoque es eficiente en términos de memoria, ya que no es necesario cargar y tokenizar todo el conjunto de datos de una sola vez.

La función se define de la siguiente manera:

In [ ]:
def yield_tokens(data_iter):
    # Función generadora para producir tokens a partir del conjunto de datos
    for _, text in data_iter:
        yield tokenizer(text)


Al invocar esta función, se crea un iterador que, al recorrerlo, devuelve listas de tokens correspondientes a cada texto. Por ejemplo, al crear un iterador con:


In [ ]:
# Crea un iterador llamado mi_iterator utilizando la función generadora yield_tokens
mi_iterator = yield_tokens(dataset)

y llamar a `next(my_iterator)`, se obtendrá la lista de tokens para el primer texto del dataset.

In [ ]:
# Obtiene el siguiente elemento del iterador mi_iterator
next(mi_iterator)

**Construcción del vocabulario y asignación de índices**

Una vez que se han generado los tokens, el siguiente paso es construir un vocabulario que asigne un índice único a cada token. En **torchtext**, esto se logra mediante la función `build_vocab_from_iterator`, que recibe como entrada un iterador de tokens. 

La función **```build_vocab_from_iterator```**, cuando se aplica a una lista de tokens, asigna un índice único a cada token basado en su posición en el vocabulario. Estos índices sirven como una forma de representar los tokens en un formato numérico que puede ser procesado fácilmente por modelos de aprendizaje automático.

Por ejemplo, dado un vocabulario con tokens `["apple", "banana", "orange"]`, los índices correspondientes podrían ser `[0, 1, 2]`, donde "apple" se representa con el índice 0, "banana" con el 1, y "orange" con el 2.
 
El proceso de tokenización se realiza de forma perezosa, lo que significa que el siguiente texto tokenizado se genera solo cuando es necesario, ahorrando memoria y recursos computacionales.

Un aspecto fundamental es que, durante la construcción del vocabulario, se pueden definir **tokens especiales** que tienen un significado específico en tareas de NLP. 



In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

**Fuera del vocabulario (OOV)**

En el mundo del procesamiento del lenguaje, es común que durante la inferencia o cuando se reciben nuevos datos, aparezcan palabras que no formaron parte del conjunto de entrenamiento o del vocabulario construido. Estas palabras se denominan **fuera del vocabulario (OOV)**. El uso de un token especial como `"<unk>"` permite al modelo manejar estas situaciones de manera robusta.

Por ejemplo, si durante el entrenamiento el vocabulario contiene la palabra "apple" pero no se ha visto "pineapple", al tokenizar un nuevo texto que contenga "pineapple", el vocabulario devolverá el índice correspondiente a `"<unk>"`. Esto garantiza que el modelo tenga una forma consistente de tratar palabras no reconocidas, evitando así que se generen índices erróneos o se interrumpa el procesamiento.

El manejo OOV es fundamental en escenarios reales donde el lenguaje es dinámico y las palabras nuevas pueden aparecer en cualquier momento. Al asignar un índice fijo para `"<unk>"`, el modelo aprende a generalizar mejor y a manejar la variabilidad del lenguaje.



**Ejemplo de tokenización y construcción del vocabulario**

Se define una función que toma un iterador y devuelve una oración tokenizada junto con sus índices:

In [ ]:
def get_tokenized_sentence_and_indices(iterator):
    tokenized_sentence = next(iterator)  # Obtener la siguiente oración tokenizada
    token_indices = [vocab[token] for token in tokenized_sentence]  # Obtener los índices de los tokens
    return tokenized_sentence, token_indices

tokenized_sentence, token_indices = get_tokenized_sentence_and_indices(my_iterator)
next(my_iterator)

print("Oracion tokenizadas:", tokenized_sentence)
print("Índices de tokens:", token_indices)

Se muestra un ejemplo donde se tokenizan varias líneas de texto, se añaden tokens especiales y se construye un vocabulario más elaborado:


In [ ]:
lineas = ["IBM taught me tokenization", 
         "Special tokenizers are ready and they will blow your mind", 
         "just saying hi!"]

special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')

tokens = []
max_length = 0

for linea in lineas:
    tokenized_line = tokenizer_en(linea)
    tokenized_line = ['<bos>'] + tokenized_line + ['<eos>']
    tokens.append(tokenized_line)
    max_length = max(max_length, len(tokenized_line))

for i in range(len(tokens)):
    tokens[i] = tokens[i] + ['<pad>'] * (max_length - len(tokens[i]))

print("Líneas después de agregar tokens especiales:\n", tokens)

# Construir vocabulario sin unk_init
vocab = build_vocab_from_iterator(tokens, specials=['<unk>'])
vocab.set_default_index(vocab["<unk>"])

# Vocabulario e IDs de tokens
print("Vocabulario:", vocab.get_itos())
print("ID de tokens para 'tokenizacion':", vocab.get_stoi())

Desglosemos la salida:

1. **Tokens especiales**:
- Token: "`<unk>`", Índice: 0: `<unk>` significa "desconocido" y representa las palabras que no se vieron durante la construcción del vocabulario, usualmente durante la inferencia en texto nuevo.
- Token: "`<pad>`", Índice: 1: `<pad>` es un token de "relleno" utilizado para hacer que las secuencias de palabras tengan la misma longitud al agruparlas en lotes.
- Token: "`<bos>`", Índice: 2: `<bos>` es el acrónimo de "inicio de secuencia" y se usa para denotar el comienzo de una secuencia de texto.
- Token: "`<eos>`", Índice: 3: `<eos>` es el acrónimo de "fin de secuencia" y se usa para denotar el final de una secuencia de texto.

2. **Tokens de palabras**:
El resto de los tokens son palabras o signos de puntuación extraídos de las oraciones proporcionadas, cada uno asignado a un índice único:
- Token: "IBM", Índice: 5
- Token: "taught", Índice: 16
- Token: "me", Índice: 12
    ... y así sucesivamente.
    
3. **Vocabulario**:
Representa el número total de tokens en las oraciones sobre las cuales se construyó el vocabulario.
    
4. **IDs de tokens para 'tokenization'**:
Representa los IDs de tokens asignados en el vocabulario donde un número representa su presencia en la oración.

Una vez que se han procesado las líneas, se procede a construir el vocabulario:

```python
vocab = build_vocab_from_iterator(tokens, specials=['<unk>'])
vocab.set_default_index(vocab["<unk>"])
```

- Aquí se construye un vocabulario que incluye los tokens especiales y los tokens provenientes de las líneas de texto.
- El vocabulario asigna índices a cada token, y cualquier token no presente en el vocabulario devolverá el índice de `"<unk>"`.

Se puede visualizar el vocabulario y comprobar la asignación de índices con las siguientes líneas:

```python
print("Vocabulario:", vocab.get_itos())
print("ID de tokens para 'tokenizacion':", vocab.get_stoi())
```

- La función `get_itos()` (ítems-to-string) devuelve una lista de tokens ordenada según su índice.
- La función `get_stoi()` (string-to-index) devuelve un diccionario en el que cada token se mapea a su índice correspondiente. Por ejemplo, el token `"tokenization"` tendrá un índice si fue incluido en el conjunto de tokens durante la construcción del vocabulario.

**Tokenización y conversión de una nueva línea con manejo de OOV**

Es frecuente que, una vez entrenado el modelo, se reciba un nuevo texto que contenga palabras no vistas durante la construcción del vocabulario. En el siguiente fragmento se muestra cómo se tokeniza una nueva línea y se manejan estas palabras desconocidas:

In [ ]:
nueva_linea = "I learned about embeddings and attention mechanisms."

# Tokenizar la nueva línea
tokenized_new_line = tokenizer_en(nueva_linea)
tokenized_new_line = ['<bos>'] + tokenized_new_line + ['<eos>']

# Rellenar la nueva línea para que coincida con la longitud máxima de las líneas anteriores
new_line_padded = tokenized_new_line + ['<pad>'] * (max_length - len(tokenized_new_line))

# Convertir tokens a IDs y manejar palabras desconocidas
new_line_ids = [vocab[token] if token in vocab else vocab['<unk>'] for token in new_line_padded]

# Ejemplo de uso
print("Id de tokens para nueva linea:", new_line_ids)

Desglosemos la salida:

1. **Tokens especiales**:
- Token: "`<unk>`", Índice: 0: `<unk>` significa "desconocido" y representa las palabras que no se vieron durante la construcción del vocabulario, usualmente durante la inferencia en texto nuevo.
- Token: "`<pad>`", Índice: 1: `<pad>` es un token de "relleno" utilizado para hacer que las secuencias de palabras tengan la misma longitud al agruparlas en lotes.
- Token: "`<bos>`", Índice: 2: `<bos>` es el acrónimo de "inicio de secuencia" y se usa para denotar el comienzo de una secuencia de texto.
- Token: "`<eos>`", Índice: 3: `<eos>` es el acrónimo de "fin de secuencia" y se usa para denotar el final de una secuencia de texto.

2. El token **`and`** es reconocido en la oración y se le asigna **`token_id` - 7**.


El manejo de OOV es esencial para garantizar que el modelo pueda procesar cualquier entrada, incluso si contiene términos nuevos o errores tipográficos. El uso consistente de `"<unk>"` permite que la red neuronal aprenda a interpretar estas situaciones de forma coherente.


#### Ejercicio: Tokenización comparativa de texto y análisis de rendimiento
- Objetivo: Evaluar y comparar las capacidades de tokenización de cuatro librerías de NLP diferentes (`nltk`, `spaCy`, `BertTokenizer` y `XLNetTokenizer`) analizando la frecuencia de las palabras tokenizadas y midiendo el tiempo de procesamiento de cada herramienta usando `datetime`.
- El texto para la tokenización es el siguiente:


In [ ]:
texto = """
Going through the world of tokenization has been like walking through a huge maze made of words, symbols, and meanings. Each turn shows a bit more about the cool ways computers learn to understand our language. And while I'm still finding my way through it, the journey’s been enlightening and, honestly, a bunch of fun.
Eager to see where this learning path takes me next!"
"""

# Contar y mostrar tokens y su frecuencia
#from collections import Counter
#def show_frequencies(tokens, method_name):
#    print(f"{method_name} Token Frequencies: {dict(Counter(tokens))}\n")

In [ ]:
## Tu respuesta
